In [1]:
import os
import nltk
import re
# import nltk libraries for preprocessing
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import json
import math
import numpy as np
from tqdm import tqdm
import pickle
os.chdir('News Articles')
nltk.download('wordnet')


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ussin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [60]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [51]:
# make a preprocess function that takes in a string and returns a list of words
def preprocess(text):
    # remove punctuation
    text = re.sub(r'[^\w\s]','',text)
    # make all words lowercase
    text = text.lower()
    # split into a list of words
    text = text.split()
    # remove stopwords
    text = [word for word in text if word not in stopwords.words('english')]
    # remove words with length==1
    text = [word for word in text if len(word)>1]
    # lemmatize words
    lemmatizer = WordNetLemmatizer()
    text = [lemmatizer.lemmatize(word) for word in text]
    return text

In [52]:
# open files in the directory each file contains add en extra section in dictionary of the articles with the key as 'article-id' and vallue as the file name+article number
# for example if the file name is 'abc-news.json' and the article number is 1 then the key value pair will be 'article-id':'abc-news/1'
# append this key in the dictionary of each article in the file and update the file with the new dictionary as json
def update():
    for file in os.listdir():
        if len(file.split('_')) > 1:
                continue
        with open(file, 'r') as f:
            data_dict = json.load(f)
        for i in range(len(data_dict['articles'])):
            data_dict['articles'][i]['article-id'] = file.split('.')[0]+'/'+str(i+1)
            f.close()
        with open(file, 'w') as f:
            json.dump(data_dict, f, indent = 4)
            f.close()

In [ ]:
# update()

In [53]:
# now we preprocess the text in each article and store it in a dictionary with key as the article-id and value as the list of words
# send the author title description and content of each article to the preprocess function and store the list of words in a dictionary
# with key as the article-id and value as the list of words in a new file called 'filename_preprocessed.json'
def preprocess_articles():
    preprocessed_dict = {}
    for file in os.listdir():
        print(file)
        with open(file, 'r') as f:
            if len(file.split('_')) > 1:
                continue
            data_dict = json.load(f)
            for i in range(len(data_dict['articles'])):
                # only preprocess article section that are not empty
                preprocessed_dict[data_dict['articles'][i]['article-id']] = []
                if data_dict['articles'][i]['content'] != None:
                    preprocessed_dict[data_dict['articles'][i]['article-id']].extend(preprocess(data_dict['articles'][i]['content']))
                if data_dict['articles'][i]['title'] != None:
                    preprocessed_dict[data_dict['articles'][i]['article-id']].extend(preprocess(data_dict['articles'][i]['title']))
                if data_dict['articles'][i]['description'] != None:
                    preprocessed_dict[data_dict['articles'][i]['article-id']].extend(preprocess(data_dict['articles'][i]['description']))
                if data_dict['articles'][i]['author'] != None:
                    preprocessed_dict[data_dict['articles'][i]['article-id']].extend(preprocess(data_dict['articles'][i]['author']))
            f.close()
    with open('preprocessed_files.json', 'w') as f:
        json.dump(preprocessed_dict, f, indent = 4)
        f.close()

In [101]:
# now we create a dictionary with key as the document id and value as along with 
# the link of the article only and no other information and save the whole dict as map.json
def create_map():
    map_dict = {}
    queries = ['crypto', 'tech', 'business', 'spacex', 'tesla', 'politics', 'AI', 'environment', 'youth activism', 'sustainable', 'climate change', 'social justice', 'women rights', 'sports', 'entertainment', 'cricket', 'comedy', 'chess']
    for file in os.listdir():
        if file.split('.')[0] not in queries:
            continue
        with open(file, 'r') as f:
            data_dict = json.load(f)
            for i in range(len(data_dict['articles'])):
                map_dict[data_dict['articles'][i]['article-id']] = data_dict['articles'][i]['url']
    with open('maplink.json', 'w') as f:
        json.dump(map_dict, f, indent = 4)
        f.close()

In [102]:
create_map()

In [ ]:
# create a dictionary for term frequency that has key as the article-id and value as a dictionary with key as the word and value as the frequency of the word in the article
def create_tf():
    tf_dict = {}
    with open('preprocessed_files.json', 'r') as f:
        preprocessed_dict = json.load(f)
        for key in preprocessed_dict.keys():
            tf_dict[key] = {}
            for word in preprocessed_dict[key]:
                if word in tf_dict[key].keys():
                    tf_dict[key][word] += 1
                else:
                    tf_dict[key][word] = 1
    with open('tf.json', 'w') as f:
        json.dump(tf_dict, f, indent = 4)
        f.close()

create_tf()

In [2]:
# create posting list that has key as the word and value as a list of article-ids in which the word is present and the number of documents in which the word is present
def create_posting_list():
    posting_list = {}
    with open('preprocessed_files.json', 'r') as f:
        preprocessed_dict = json.load(f)
        for key in preprocessed_dict.keys():
            for word in preprocessed_dict[key]:
                if word in posting_list.keys():
                    posting_list[word][0].append(key)
                    posting_list[word][1] += 1
                else:
                    posting_list[word] = [[key], 1]
    with open('posting_list.json', 'w') as f:
        json.dump(posting_list, f, indent = 4)
        f.close()

# create idf dict
def create_idf_dict(no_of_docs, posting_list):
    idf_dict = {}
    for token in posting_list:
        idf_dict[token] = math.log(no_of_docs / posting_list[token][1]+1)
    with open('idf.json', 'w') as f:
        json.dump(idf_dict, f, indent = 4)
        f.close()


# create_posting_list()
posting_list = json.load(open('posting_list.json', 'r'))
number_of_docs = len(json.load(open('preprocessed_files.json', 'r')))
# create_idf_dict(number_of_docs, posting_list)

In [3]:
# create the tf-idf matrix for each document and 
# Weighting Scheme TF Weight
# Binary 0,1
# Raw count f(t,d)
# Term frequency f(t,d)/Pf(t‘, d)
# Log normalization log(1+f(t,d))
# Double normalization 0.5+0.5*(f(t,d)/ max(f(t‘,d))

def create_tf_idf_matrix_binary(no_of_docs, tf_dict, idf_dict):
    vocab_size = len(idf_dict)
    tf_idf_matrix = np.zeros((no_of_docs, vocab_size))
    # check for all terms in the idf_dict, if it is present in the tf_dict of the document, then set the value to 1*idf
    for i, filename in enumerate(tqdm(tf_dict.keys())):
        for j, token in enumerate(idf_dict):
            if token in tf_dict[filename]:
                tf_idf_matrix[i][j] = 1*idf_dict[token]
                
    return tf_idf_matrix

def create_tf_idf_matrix_raw_count(no_of_docs, tf_dict, idf_dict):
    vocab_size = len(idf_dict)
    tf_idf_matrix = np.zeros((no_of_docs, vocab_size))
    for i, filename in enumerate(tqdm(tf_dict.keys())):
        for j, token in enumerate(idf_dict):
            if token in tf_dict[filename]:
                tf_idf_matrix[i][j] = tf_dict[filename][token]*idf_dict[token]
                
    return tf_idf_matrix

def create_tf_idf_matrix_term_frequency(no_of_docs, tf_dict, idf_dict):
    vocab_size = len(idf_dict)
    tf_idf_matrix = np.zeros((no_of_docs, vocab_size))
    for i, filename in enumerate(tqdm(tf_dict.keys())):
        for j, token in enumerate(idf_dict):
            if token in tf_dict[filename]:
                tf_idf_matrix[i][j] = (tf_dict[filename][token]/len(tf_dict[filename]))*idf_dict[token]

    return tf_idf_matrix

def create_tf_idf_matrix_log_normalization(no_of_docs, tf_dict, idf_dict):
    vocab_size = len(idf_dict)
    tf_idf_matrix = np.zeros((no_of_docs, vocab_size))
    for i, filename in enumerate(tqdm(tf_dict.keys())):
        for j, token in enumerate(idf_dict):
            if token in tf_dict[filename]:
                tf_idf_matrix[i][j] = math.log(1+tf_dict[filename][token])*idf_dict[token]
    
    return tf_idf_matrix

def create_tf_idf_matrix_double_normalization(no_of_docs, tf_dict, idf_dict):
    vocab_size = len(idf_dict)
    tf_idf_matrix = np.zeros((no_of_docs, vocab_size))
    for i, filename in enumerate(tqdm(tf_dict.keys())):
        max_tf = 0
        for token in tf_dict[filename]:
            if tf_dict[filename][token] > max_tf:
                max_tf = tf_dict[filename][token]
        for j, token in enumerate(idf_dict):
            if token in tf_dict[filename]:
                tf_idf_matrix[i][j] = 0.5+0.5*(tf_dict[filename][token]/max_tf)*idf_dict[token]
    return tf_idf_matrix


In [4]:
# create tf-idf matrix
binary_tf_idf_matrix = create_tf_idf_matrix_binary(number_of_docs, json.load(open('tf.json', 'r')), json.load(open('idf.json', 'r')))
raw_count_tf_idf_matrix = create_tf_idf_matrix_raw_count(number_of_docs, json.load(open('tf.json', 'r')), json.load(open('idf.json', 'r')))
term_frequency_tf_idf_matrix = create_tf_idf_matrix_term_frequency(number_of_docs, json.load(open('tf.json', 'r')), json.load(open('idf.json', 'r')))
log_normalization_tf_idf_matrix = create_tf_idf_matrix_log_normalization(number_of_docs, json.load(open('tf.json', 'r')), json.load(open('idf.json', 'r')))
double_normalization_tf_idf_matrix = create_tf_idf_matrix_double_normalization(number_of_docs, json.load(open('tf.json', 'r')), json.load(open('idf.json', 'r')))

100%|██████████| 1800/1800 [00:04<00:00, 449.15it/s]


In [5]:
# save the similarity matrix in a pickle file
with open('binary_tf_idf_matrix.pkl', 'wb') as f:
    pickle.dump(binary_tf_idf_matrix, f)
with open('raw_count_tf_idf_matrix.pkl', 'wb') as f:
    pickle.dump(raw_count_tf_idf_matrix, f)
with open('term_frequency_tf_idf_matrix.pkl', 'wb') as f:
    pickle.dump(term_frequency_tf_idf_matrix, f)
with open('log_normalization_tf_idf_matrix.pkl', 'wb') as f:
    pickle.dump(log_normalization_tf_idf_matrix, f)
with open('double_normalization_tf_idf_matrix.pkl', 'wb') as f:
    pickle.dump(double_normalization_tf_idf_matrix, f)

In [27]:
# load the tweetdata csv file
tweet_data = pd.read_csv('tweetdata.csv', encoding = 'latin1')
tweet_data.head

<bound method NDFrame.head of       Unnamed: 0                       Date      User  \
0              0  2023-03-23 21:55:32+00:00  elonmusk   
1              1  2023-03-23 21:54:15+00:00  elonmusk   
2              2  2023-03-23 21:31:38+00:00  elonmusk   
3              3  2023-03-23 19:44:49+00:00  elonmusk   
4              4  2023-03-23 09:31:25+00:00  elonmusk   
...          ...                        ...       ...   
1995        1995  2022-12-26 17:58:30+00:00  elonmusk   
1996        1996  2022-12-26 17:56:10+00:00  elonmusk   
1997        1997  2022-12-26 17:33:10+00:00  elonmusk   
1998        1998  2022-12-26 17:29:43+00:00  elonmusk   
1999        1999  2022-12-26 16:42:20+00:00  elonmusk   

                                                  Tweet hashtag  \
0                                     @Chad_Hurley ð      []   
1     @BillyM2k ð¶ Always look on the bright side ...      []   
2             Twitter Verified now available worldwide!      []   
3                

In [46]:
# pick up the columnn text and convert it to a list
tweet = tweet_data['Tweet'].tolist()

In [48]:
# append all the tweeets space seperated to make a paragraph
paragraph = ' '.join(tweet)

In [93]:
tweet_corpus = preprocess(paragraph)

In [94]:
# idf_dict is the vocabulary vector create a query vector such that if the word is present in the query, then set the value to 1, where query vector is tweet_cotpus
idf_dict = json.load(open('idf.json', 'r'))
query_vector = []
for token in idf_dict:
    if token in tweet_corpus:
        query_vector.append(1)
    else:
        query_vector.append(0)

In [95]:
# create a similarity matrix for the query vector and the tf-idf matrix
binary_tf_idf_matrix = pickle.load(open('binary_tf_idf_matrix.pkl', 'rb'))
raw_count_tf_idf_matrix = pickle.load(open('raw_count_tf_idf_matrix.pkl', 'rb'))
term_frequency_tf_idf_matrix = pickle.load(open('term_frequency_tf_idf_matrix.pkl', 'rb'))
log_normalization_tf_idf_matrix = pickle.load(open('log_normalization_tf_idf_matrix.pkl', 'rb'))
double_normalization_tf_idf_matrix = pickle.load(open('double_normalization_tf_idf_matrix.pkl', 'rb'))

binary_similarity_matrix = cosine_similarity(binary_tf_idf_matrix, [query_vector])
raw_count_similarity_matrix = cosine_similarity(raw_count_tf_idf_matrix, [query_vector])
term_frequency_similarity_matrix = cosine_similarity(term_frequency_tf_idf_matrix, [query_vector])
log_normalization_similarity_matrix = cosine_similarity(log_normalization_tf_idf_matrix, [query_vector])
double_normalization_similarity_matrix = cosine_similarity(double_normalization_tf_idf_matrix, [query_vector])

In [96]:
# recommend top 10 articles
def recommend_top_10_articles(similarity_matrix):
    top_10 = np.argsort(similarity_matrix, axis=0)[-10:][::-1]
    top_10 = top_10.reshape(10)
    return top_10


binary_top_10 = recommend_top_10_articles(double_normalization_similarity_matrix)

In [110]:
# use the maplink.json to recommend top 10 news artiles
maplink = json.load(open('maplink.json', 'r'))
for i, (key, value) in enumerate(maplink.items()):
    if i in binary_top_10:
        print(key)
        print(value)

chess/57
https://www.slashfilm.com/1229380/the-crown-anchor-pub-from-ted-lasso-is-up-for-rent-on-airbnb/
crypto/59
https://www.businessinsider.com/elon-musk-history-of-pets-dogs-cats-hobbes-floki-schrodinger-2023-3
spacex/34
https://www.wired.com/story/the-mystery-vehicle-at-the-heart-of-teslas-new-master-plan/
spacex/55
https://gizmodo.com/elon-musk-twitter-spacex-morgan-stanley-tesla-1850199556
sustainable/31
https://www.wired.com/story/the-mystery-vehicle-at-the-heart-of-teslas-new-master-plan/
tech/16
https://www.engadget.com/the-apple-watch-se-drops-to-its-lowest-price-yet-plus-the-rest-of-the-weeks-best-tech-deals-164528358.html
tesla/2
https://www.wired.com/story/the-mystery-vehicle-at-the-heart-of-teslas-new-master-plan/
tesla/7
https://www.businessinsider.com/elon-musk-buy-silicon-valley-bank-tesla-investor-no-thanks-2023-3
tesla/72
https://www.businessinsider.com/elon-musk-says-ai-dangerous-technology-needs-regulating-2023-3
youth activism/59
https://www.dazeddigital.com/life